### Inlezen van de benodigde packages

In [1]:
import pandas as pd
import numpy as np
import re

###### In onderstaande cel kunnen de aantal rijen/kolommen die max. geprint worden aangepast worden. Pas op met 'None', als de dataset te groot is loopt de notebook vast 

In [2]:
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows    = 50
pd.set_option('max_colwidth', 100)

### Inlezen van de CBS data 

In [3]:
data_cbs = pd.read_csv("/Users/vcgorka/Desktop/JADS/Bootcamp/open_data/buurtcode_postcode.csv", delimiter= ';')

###### Daar alleen de buurtcode voldoende is kunnen we de overige kolommen verwijderen. Daarnaast verwijderen we alle dubbele rijen (die ontstaan zijn door het verwijderen van de huisnummers. De "keep" waarde =  'first' behoudt alleen de eerste, 'last' alleen de laatste en 'false' verwijderd alles

In [4]:
data_cbs = data_cbs.drop(['Huisnummer'], 1)
data_cbs = data_cbs.drop(['Wijk2017']  , 1)
data_cbs = data_cbs.drop(['Gem2017']   , 1)

data_cbs.drop_duplicates(subset=['PC6'], keep='first', inplace=True)

###### In onderstaande cell wordt een dictionary object gemaakt. Het voordeel van zo'n object is dat je snel een waarde kan opzoeken op basis van een key. Door alle postcodes te koppelen aan een buurtcode kan snel een link gelegd worden tussen deze twee waarde. Dit versnelt later op het proces om te bepalen welke postcode bij welke buurtcode hoort. 

In [5]:
d = dict()

for index, row in data_cbs.iterrows():
    pc = row['PC6']
    bc = row['Buurt2017']
    
    if pc in d.keys():
        if bc not in d[pc]:
            d[pc].append(bc)
    else:
        d[pc] = [bc]

###### Bepaal welke records over meerdere postcode gebieden lopen en behoudt alleen de gelijke cijferpostcode gebieden 

In [6]:
'''
In deze methode worden de PC4 waarde vergelijken, als de PC4 waarde gelijk is wordt '-1' geretouneerd, 
als de PC4 verschilt wordt het absolute verschil terug gegeven.
'''
def overlappende_postcodes(row):
    postcode_1 = row['zipcode_from']
    postcode_2 = row['zipcode_to']
    
    cijfers_postcode_1 = int(re.split('(\d+)',postcode_1)[1])
    cijfers_postcode_2 = int(re.split('(\d+)',postcode_2)[1])
    
    if(cijfers_postcode_1 == cijfers_postcode_2):
        return -1
    else:
        return np.abs(cijfers_postcode_1-cijfers_postcode_2)

In [7]:
'''
In deze methode worden de postcodes gesplits, de range die gegeven wordt, wordt gebruikt als input
om de tussenliggende postcodes te bepalen. 
'''
def split_postcodes(row):
    postcode_1 = row['zipcode_from']
    postcode_2 = row['zipcode_to']

    cijfers_postcode_1 = int(re.split('(\d+)',postcode_1)[1])
    cijfers_postcode_2 = int(re.split('(\d+)',postcode_2)[1])

    letter1_postcode_1 = (re.split('(\d+)',postcode_1)[2])[0]
    letter2_postcode_1 = (re.split('(\d+)',postcode_1)[2])[1]
    letter1_postcode_2 = (re.split('(\d+)',postcode_2)[2])[0]
    letter2_postcode_2 = (re.split('(\d+)',postcode_2)[2])[1]

    letter_1_range = letter_range(letter1_postcode_1, letter1_postcode_2)
    letter_2_range = letter_range(letter2_postcode_1, letter2_postcode_2)

    postcodes = []

    if(len(letter_1_range) == 1):
        for j in letter_2_range:
            postcodes.append(str(cijfers_postcode_1) + letter_1_range[0] + j)
    else:
        for i in range(len(letter_1_range)):
            if i == 0:
                for j in letter_range(letter2_postcode_1, "Z"):
                    postcodes.append(str(cijfers_postcode_1) + letter_1_range[i] + j)
            elif i == len(letter_1_range)-1:
                for j in letter_range("A", letter2_postcode_2):
                    postcodes.append(str(cijfers_postcode_1) + letter_1_range[i]+ j)
            else:
                for j in letter_range("A", "Z"):
                    postcodes.append(str(cijfers_postcode_1) + letter_1_range[i]+ j)   

    return postcodes

In [8]:
'''
Methode om alle tussenliggende letters in een range te bepallen
'''
def letter_range(l1, l2):
    letters = [l1]
    einde = True
    
    while letters[-1] is not l2:
        if(letters[-1] == "Z"):
            letter = "A"
            letters.append(letter)
        else:
            letter = chr(ord(letters[-1])+ 1)
            letters.append(letter)    
    return letters

In [9]:
'''
In deze methode wordt gebruik gemaakt van de dictionary d die we eerder aangemaakt hebben.
Hier wordt per postcode de corronderende postcode opgezocht.
'''
def koppel_buurt_code(row):
    postcodes = row['postcodes']
    wijkcodes = []
    
    
    for pc in postcodes:
        try: 
            wijkcode = d[pc][0]
            if wijkcode not in wijkcodes:
                wijkcodes.append(wijkcode)
        except:
            wijkcodes = wijkcodes
    
    if(len(wijkcodes) == 0):
        return 0
    else:
        return wijkcodes

In [10]:
'''
Telt het aantal buurtcodes, die hebben wij gebruikt om snel te bepalen welke datalijnen
aan meer dan 1 buurtcode toegeschreven zijn
'''
def tel_buurtcodes(row):
    try:
        wijkcodes = row['wijkcodes']
        return len(wijkcodes)
    except:
        return 0

In [11]:
'''
Telt het aantal postcodes
'''
def tel_postcodes(row):
    try:
        postcodes = row['postcodes']
        return len(postcodes)
    except:
        return 0

In [12]:
'''
Geeft de laatste wijk/buurtcode terug uit de lijst met buurtcodes per datapunt
'''
def wijkcodes(row):
    return row['wijkcodes'][-1]

In [13]:
'''
Met deze methode is het totaal stroomverbruik afgeleidt uit het jaarlijkse
verbuik samen met het delivery percentage - het percentage dat aangeeft welk
deel het geleverde jaarlijks totaal is van de stroomleverancier, en daarmee dus 
welk deel niet door hen geleverd is maar door eigen stroomopwekking
'''
def bereken_totaal(row):
    netto_verbruik   = row['annual_consume']
    netto_percentage = row['delivery_perc']
    
    if(netto_percentage != 0):
        return (netto_verbruik / (netto_percentage / 100))
    else:
        return 0

In [14]:
def prepare_data(data_path, provider):
    data = pd.read_csv(data_path)

    data['postcode_1'] = 0
    data['postcode_1'] = data.apply(overlappende_postcodes, axis=1)
    
    data = data[data['postcode_1'] < 0]
    
    data['postcodes'] = 0
    data['postcodes'] = data.apply(split_postcodes, axis = 1)
    
    data['wijkcodes'] = 0
    data['wijkcodes'] = data.apply(koppel_buurt_code, axis = 1)
    
    data['aantal_wijkcodes'] = 0
    data['aantal_wijkcodes'] = data.apply(tel_buurtcodes, axis = 1)
    
    data['aantal_postcodes'] = 0
    data['aantal_postcodes'] = data.apply(tel_postcodes, axis = 1)
    
    data = data[data['aantal_postcodes'] > 0]
    data = data[data['aantal_wijkcodes'] > 0]

    data['kwh_laag_tarief']         = ((data['annual_consume'] / 100) * data['annual_consume_lowtarif_perc'])
    data['kwh_hoog_tarief']         = data['annual_consume'] - data['kwh_laag_tarief']
    data['actieve_connecties']      = data['num_connections'] * (data['perc_of_active_connections'] / 100)
    data['connecties_m_smartmeter'] = data['num_connections'] * (data['smartmeter_perc'] / 100)
    data['total_consumption']       = data.apply(bereken_totaal, axis = 1)
    data['energy_to_grid']          = data['total_consumption'] - data['annual_consume']
    
    data['wijkcode'] = 0
    data['wijkcode'] = data.apply(wijkcodes, axis = 1)
    
    data = data.drop(['net_manager', 'street', 'zipcode_from', 'zipcode_to', 
                     'zipcode_from', 'zipcode_to', 'city',
                     'postcode_1', 'postcodes', 'wijkcodes', 
                      'aantal_wijkcodes','aantal_postcodes', 'type_conn_perc',
                     'delivery_perc', 'perc_of_active_connections',
                     'annual_consume_lowtarif_perc', 'smartmeter_perc'], 1)
    
    data['purchase_area'] = provider
    
    aggregations = { 
        'purchase_area': 'first', 
        'num_connections': 'sum', 
        'type_of_connection': 'first', 
        'annual_consume': 'sum',  
        'kwh_laag_tarief': 'sum',
        'kwh_hoog_tarief': 'sum', 
        'actieve_connecties': 'sum', 
        'connecties_m_smartmeter': 'sum',
        'total_consumption': 'sum', 
        'energy_to_grid': 'sum', 
    }
    
    data = data.groupby(['wijkcode']).agg(aggregations)
    
    data['actieve_connecties']        =   data['actieve_connecties'].astype(int)
    data['total_consumption']         =   data['total_consumption'].astype(int)
    data['energy_to_grid']            =   data['energy_to_grid'].astype(int)
    data['kwh_laag_tarief']           =   data['kwh_laag_tarief'].astype(int)
    data['kwh_hoog_tarief']           =   data['kwh_hoog_tarief'].astype(int)
    data['connecties_m_smartmeter']   =   data['connecties_m_smartmeter'].astype(int)
    
    return data

###### Data methode om data te prepareren zodat deze aan elkaar gekoppeld kan worden

In [15]:
def verander_kolomnamen(df, jaar):
    kolomnamen = df.columns
    nieuwe_namen = []
    
    for kolom in kolomnamen:
        label = kolom + '_' + jaar
        nieuwe_namen.append(label)
    
    df.columns = nieuwe_namen
    
    return df

###### inlezen data en veranderen kolomnamen

In [16]:
prefix = "/Users/vcgorka/Desktop/JADS/bootcamp/data/"

In [17]:
filename = prefix + "liander_electricity_01012010.csv"
%time data_2010 = verander_kolomnamen(prepare_data(filename, 'Liander'), '2010')

CPU times: user 17.9 s, sys: 444 ms, total: 18.4 s
Wall time: 18.5 s


In [18]:
filename = prefix + "liander_electricity_01012011.csv"
%time data_2011 = verander_kolomnamen(prepare_data(filename, 'Liander'), '2011')

CPU times: user 18.2 s, sys: 437 ms, total: 18.6 s
Wall time: 18.7 s


In [19]:
filename = prefix + "liander_electricity_01012012.csv"
%time data_2012 = verander_kolomnamen(prepare_data(filename, 'Liander'), '2012')

CPU times: user 18.6 s, sys: 451 ms, total: 19 s
Wall time: 19.1 s


In [20]:
filename = prefix + "liander_electricity_01012013.csv"
%time data_2013 = verander_kolomnamen(prepare_data(filename, 'Liander'), '2013')

CPU times: user 18.6 s, sys: 454 ms, total: 19.1 s
Wall time: 19.1 s


In [21]:
filename = prefix + "liander_electricity_01012014.csv"
%time data_2014 = verander_kolomnamen(prepare_data(filename, 'Liander'), '2014')

CPU times: user 17.8 s, sys: 387 ms, total: 18.2 s
Wall time: 18.3 s


In [22]:
filename = prefix + "liander_electricity_01012015.csv"
%time data_2015 = verander_kolomnamen(prepare_data(filename, 'Liander'), '2015')

CPU times: user 17.5 s, sys: 342 ms, total: 17.9 s
Wall time: 17.9 s


In [23]:
filename = prefix + "liander_electricity_01012016.csv"
%time data_2016 = verander_kolomnamen(prepare_data(filename, 'Liander'), '2016')

CPU times: user 18.1 s, sys: 343 ms, total: 18.4 s
Wall time: 18.5 s


In [24]:
filename = prefix + "liander_electricity_01012017.csv"
%time data_2017 = verander_kolomnamen(prepare_data(filename, 'Liander'), '2017')

CPU times: user 18 s, sys: 353 ms, total: 18.3 s
Wall time: 18.3 s


In [25]:
filename = prefix + "liander_electricity_01012018.csv"
%time data_2018 = verander_kolomnamen(prepare_data(filename, 'Liander'), '2018')

CPU times: user 17.9 s, sys: 350 ms, total: 18.3 s
Wall time: 18.3 s


###### Data ingelezen en merging

In [26]:
result = pd.concat([data_2010, data_2011], axis=1, sort=False)
result = pd.concat([result, data_2012], axis=1, sort=False)
result = pd.concat([result, data_2013], axis=1, sort=False)
result = pd.concat([result, data_2014], axis=1, sort=False)
result = pd.concat([result, data_2015], axis=1, sort=False)
result = pd.concat([result, data_2016], axis=1, sort=False)
result = pd.concat([result, data_2017], axis=1, sort=False)
result = pd.concat([result, data_2018], axis=1, sort=False)

In [27]:
result.head(10)

,purchase_area_2010,num_connections_2010,type_of_connection_2010,annual_consume_2010,kwh_laag_tarief_2010,kwh_hoog_tarief_2010,actieve_connecties_2010,connecties_m_smartmeter_2010,total_consumption_2010,energy_to_grid_2010,purchase_area_2011,num_connections_2011,type_of_connection_2011,annual_consume_2011,kwh_laag_tarief_2011,kwh_hoog_tarief_2011,actieve_connecties_2011,connecties_m_smartmeter_2011,total_consumption_2011,energy_to_grid_2011,purchase_area_2012,num_connections_2012,type_of_connection_2012,annual_consume_2012,kwh_laag_tarief_2012,kwh_hoog_tarief_2012,actieve_connecties_2012,connecties_m_smartmeter_2012,total_consumption_2012,energy_to_grid_2012,purchase_area_2013,num_connections_2013,type_of_connection_2013,annual_consume_2013,kwh_laag_tarief_2013,kwh_hoog_tarief_2013,actieve_connecties_2013,connecties_m_smartmeter_2013,total_consumption_2013,energy_to_grid_2013,purchase_area_2014,num_connections_2014,type_of_connection_2014,annual_consume_2014,kwh_laag_tarief_2014,kwh_hoog_tarief_2014,actieve_connecties_2014,connecties_m_smartmeter_2014,total_consumption_2014,energy_to_grid_2014,purchase_area_2015,num_connections_2015,type_of_connection_2015,annual_consume_2015,kwh_laag_tarief_2015,kwh_hoog_tarief_2015,actieve_connecties_2015,connecties_m_smartmeter_2015,total_consumption_2015,energy_to_grid_2015,purchase_area_2016,num_connections_2016,type_of_connection_2016,annual_consume_2016,kwh_laag_tarief_2016,kwh_hoog_tarief_2016,actieve_connecties_2016,connecties_m_smartmeter_2016,total_consumption_2016,energy_to_grid_2016,purchase_area_2017,num_connections_2017,type_of_connection_2017,annual_consume_2017,kwh_laag_tarief_2017,kwh_hoog_tarief_2017,actieve_connecties_2017,connecties_m_smartmeter_2017,total_consumption_2017,energy_to_grid_2017,purchase_area_2018,num_connections_2018,type_of_connection_2018,annual_consume_2018,kwh_laag_tarief_2018,kwh_hoog_tarief_2018,actieve_connecties_2018,connecties_m_smartmeter_2018,total_consumption_2018,energy_to_grid_2018
wijkcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
340101,Liander,1042.0,3x25,276588.0,88803.0,187784.0,957.0,0.0,276588.0,0.0,Liander,1042.0,1x25,277105.0,114124.0,162980.0,996.0,0.0,277105.0,0.0,Liander,1063.0,1x25,291457.0,121038.0,170418.0,1016.0,1.0,291457.0,0.0,Liander,1068.0,1x25,294402.0,119404.0,174997.0,1019.0,27.0,294402.0,0.0,Liander,1052.0,1x25,278269.0,120661.0,157607.0,999.0,61.0,278269.0,0.0,Liander,1056.0,1x25,270747.0,120707.0,150039.0,1002.0,84.0,270747.0,0.0,Liander,1057.0,1x25,262374.0,122073.0,140300.0,1002.0,92.0,262374.0,0.0,Liander,1089.0,1x25,258326.0,120019.0,138306.0,1003.0,126.0,259255.0,929.0,Liander,1093.0,1x35,262165.0,164901.0,97263.0,1033.0,538.0,263365.0,1200.0
340102,Liander,1279.0,3x25,284222.0,75184.0,209037.0,1265.0,0.0,284222.0,0.0,Liander,1281.0,1x25,285643.0,97392.0,188250.0,1268.0,5.0,285643.0,0.0,Liander,1329.0,1x25,303411.0,101670.0,201740.0,1315.0,5.0,303411.0,0.0,Liander,1334.0,1x25,287846.0,99256.0,188589.0,1318.0,26.0,288042.0,196.0,Liander,1360.0,1x25,302275.0,126553.0,175721.0,1293.0,77.0,304066.0,1791.0,Liander,1361.0,1x25,268324.0,101270.0,167053.0,1343.0,151.0,272617.0,4293.0,Liander,1361.0,1x25,252600.0,95365.0,157234.0,1342.0,171.0,257630.0,5030.0,Liander,1385.0,1x25,248741.0,97434.0,151306.0,1343.0,195.0,255127.0,6386.0,Liander,1389.0,1x35,239052.0,161812.0,77239.0,1368.0,811.0,300396.0,61344.0
340103,Liander,1185.0,3x25,245174.0,52737.0,192436.0,1173.0,0.0,245396.0,222.0,Liander,1185.0,1x25,246861.0,71900.0,174960.0,1172.0,3.0,247078.0,217.0,Liander,1187.0,1x25,244922.0,69981.0,174940.0,1175.0,3.0,245484.0,562.0,Liander,1187.0,1x25,241374.0,69412.0,171961.0,1178.0,21.0,242252.0,878.0,Liander,1171.0,1x25,228165.0,75633.0,152531.0,1161.0,92.0,230771.0,2606.0,Liander,1172.0,1x25,224896.0,75237.0,149658.0,1162.0,103.0,228703.0,3807.0,Liander,1173.0,1x25,224372.0,76899.0,147472.0,1161.0,118.0,230756.0,6384.0,Liander,1175.0,1x25,214833.0,76422.0,138410.0,1166.0,148.0,2

###### Linken van open data 

###### cbs 

In [28]:
def get_wijkcode(row):
    try:
        return int(re.split('(\d+)',row['gwb_code'])[1]) 
    except:
        try:
            return row['gwb_code12']
        except:
            return row['gwb_code10']

In [29]:
te_verwijderen_kolommen = ['gwb_code_10', 'gwb_code_8', 'regio', 'gm_naam', 'recs',
 'gwb_code', 'ind_wbi', 'g_gas', 'g_gas_ap', 'g_gas_tw', 
 'g_gas_hw', 'g_gas_2w', 'g_gas_vw', 'g_gas_hu', 'g_gas_ko', 'p_stadsv',
 'g_wodief','g_vernoo','g_gewsek','a_bedv','a_bed_a','a_bed_bf','a_bed_gi',
 'a_bed_hj','a_bed_kl','a_bed_mn','a_bed_ru','a_pau','a_bst_b','a_bst_nb',
 'g_pau_hh','g_pau_km','a_m2w','g_afs_hp','g_afs_gs','g_afs_kv','g_afs_sc',
 'g_3km_sc','a_opp_ha','a_lan_ha','a_wat_ha','pst_mvp','pst_dekp','ste_mvs',
 'ste_oad']

In [30]:
kolomnamen = ['aantal_inwoners',
 'aantal_mannen',
 'aantal_vrouwen',
 'aantal_00_14',
 'aantal_15_24',
 'aantal_25_44',
 'aantal_45_64',
 'aantal_65_oo',
 'aantal_ongehuwd',
 'aantal_gehuwd',
 'aantal_gescheiden',
 'aantal_verweduwd',
 'aantal_westerse_allochtonen',
 'aantal_n_westerse_allochtonen',
 'aantal_marokkanen',
 'aantal_antilianen',
 'aantal_surinamers',
 'aantal_turken',
 'aantal_overige_n_westers',
 'aantal_geboren',
 'percentage_geboren',
 'aantal_doden',
 'percentage_doden',
 'aantal_huishoudens',
 'aantal_1_persoonshuishoudens',
 'aantal_huishoudens_zonder_kinderen',
 'aantal_huishoudens_met_kinderen',
 'gemiddeld_huishoudensgrootte',
 'bevolkingsdichtheid',
 'aantal_woningingen',
 'gemiddelde_woz',
 'percentage_eengezinswoningen',
 'percentage_meergezinswoningen',
 'percentage_bewoond',
 'percentage_leegstaand',
 'percentage_koopwoning',
 'percantage_huurwoning',
 'percentage_wooningbouwcorporatie',
 'percentage_overige_verhuurders',
 'percentage_eigendom_onbekend',
 'percentage_voor_2000',
 'percentage_vanaf_2000',
 'gemiddeld_elektriciteitsgebruik_totaal',
 'gemiddeld_elektriciteitsgebruik_app',
 'gemiddeld_elektriciteitsgebruik_tussenwoning',
 'gemiddeld_elektriciteitsgebruik_hoekwoning',
 'gemiddeld_elektriciteitsgebruik_2ondereenkap',
 'gemiddeld_elektriciteitsgebruik_vrijstaande_woning',
 'gemiddeld_elektriciteitsgebruik_huurwoning',
 'gemiddeld_elektriciteitsgebruik_eigenwoning',
 'aantal_inkomensontvangers',
 'gemiddeld_inkomen_per_inkomensontvanger',
 'gemiddeld_inkomen_per_inwoner',
 'percentage_met_laagste_inkomen',
 'percentage_met_hoogste_inkomen',
 'percentage_actieven',
 'percentage_huishoudens_laagste_inkomens',
 'percentage_huishoudens_hoogste_inkomens',
 'p_huishoudens_laag_inkomen',
 'p_huishoudens_onder_sociaal_minimum',
 'aantal_personen_bijstand',
 'aantal_personen_arbeidsongeschik',
 'aantal_personen_werkloos',
 'aantal_personen_aow',
 'wijkcode']

In [31]:
def inlezen_cbs_wijkdata(path, year):
    wijkdata = pd.read_excel(path)
    
    try:
        wijkdata = wijkdata[wijkdata.recs == 'Buurt']
    except:
        wijkdata = wijkdata[wijkdata.RECS == 'B']
        
    kolomnamen_lowercase = []
    for kolom in wijkdata.columns:
        kolomnamen_lowercase.append(kolom.lower())
    
    wijkdata.columns = kolomnamen_lowercase
    
    wijkdata['wijkcode'] = 0
    wijkdata['wijkcode'] = wijkdata.apply(get_wijkcode, axis = 1)

    wijkdata = wijkdata[kolommen_wijkdata]
    
    wijkdata.columns     = kolomnamen
    wijkdata             = verander_kolomnamen(wijkdata, year)

    return wijkdata

###### Data 2018 apart inlezen 

In [32]:
wijkdata             = pd.read_excel('/Users/vcgorka/Desktop/JADS/bootcamp/open_data/kwb-2018.xls')
wijkdata             = wijkdata[wijkdata.recs == 'Buurt']

wijkdata['wijkcode'] = 0
wijkdata['wijkcode'] = wijkdata.apply(get_wijkcode, axis = 1)

wijkdata             = wijkdata.drop(te_verwijderen_kolommen, axis = 1)
kolommen_wijkdata    = list(wijkdata.columns)

wijkdata.index       = wijkdata['wijkcode']
wijkdata.columns     = kolomnamen

wijkdata_2018        = verander_kolomnamen(wijkdata, '2018')
del wijkdata

###### Overige data inlezen 

In [33]:
%time wijkdata_2017 = inlezen_cbs_wijkdata('/Users/vcgorka/Desktop/JADS/bootcamp/open_data/kwb-2017.xls', '2017')
wijkdata_2017.index = wijkdata_2017['wijkcode_2017']

CPU times: user 4.12 s, sys: 93.3 ms, total: 4.21 s
Wall time: 4.32 s


In [34]:
%time wijkdata_2016 = inlezen_cbs_wijkdata('/Users/vcgorka/Desktop/JADS/bootcamp/open_data/kwb-2016.xls', '2016')
wijkdata_2016.index = wijkdata_2016['wijkcode_2016']

CPU times: user 4.42 s, sys: 135 ms, total: 4.55 s
Wall time: 4.76 s


In [35]:
%time wijkdata_2015 = inlezen_cbs_wijkdata('/Users/vcgorka/Desktop/JADS/bootcamp/open_data/kwb-2015.xls', '2015')
wijkdata_2015.index = wijkdata_2015['wijkcode_2015']

CPU times: user 4.24 s, sys: 82.5 ms, total: 4.32 s
Wall time: 4.49 s


In [36]:
result.shape

(4781, 90)

In [37]:
results = pd.concat([result, wijkdata_2015], axis=1)

In [38]:
results.shape

(13183, 155)

In [39]:
results = pd.concat([results, wijkdata_2016], axis=1)

In [40]:
results.shape

(13461, 220)

In [41]:
results = pd.concat([results, wijkdata_2017], axis=1)

In [42]:
results.shape

(13999, 285)

In [43]:
results = pd.concat([results, wijkdata_2018], axis=1)

In [44]:
results.shape

(14581, 350)

In [45]:
values = {'annual_consume_2010': -1}
results = results.fillna(value=values)

In [46]:
results = results[results['annual_consume_2010'] != -1]

In [47]:
results.shape

(4573, 350)

In [48]:
results.to_excel("/Users/vcgorka/Desktop/JADS/bootcamp/geprepareerde_dataset_groep_1.xlsx")